In [8]:
import math

## 3. Calculate emission factor

In [6]:
class EmissionFactorCalculator:

    def __init__(self ,farm_data):
        ## validate input data
        self.validate_input(farm_data)
        ## initialize instance variables with validated data
        self.data = farm_data
        self.P = farm_data['climate_data']['P']
        self.PE = farm_data['climate_data']['PE']
        self.FR_Topo = farm_data['climate_data']['FR_Topo']
        self.RF_TX = farm_data['modifiers']['RF_TX']
        self.RF_NS = farm_data['modifiers']['RF_NS']
        self.RF_till = farm_data['modifiers']['RF_Till']
        self.RF_CS = farm_data['modifiers']['RF_CS']
        self.RF_AM = farm_data['modifiers']['RF_AM']

    def validate_input(self, farm_data):
        ## data should include all required fields and should be numbers

        required_climate_keys = ['P', 'PE', 'FR_Topo']
        for key in required_climate_keys:
            if key not in farm_data['climate_data']:
                raise ValueError(f"Missing required climate data key: {key}")
            if not isinstance(farm_data['climate_data'][key], (int, float)):
                raise TypeError(f"Value for climate_data[{key}] must be a number (int or float)")
    
        required_modifiers_keys = ['RF_TX', 'RF_NS', 'RF_Till', 'RF_CS', 'RF_AM']
        for key in required_modifiers_keys:
            if key not in farm_data['modifiers']:
                raise ValueError(f"Missing required modifiers key: {key}")
            if not isinstance(farm_data['modifiers'][key], (int, float)):
                raise TypeError(f"Value for modifiers[{key}] must be a number (int or float)")    

    def calculate_ef_ct(self):
        ## calculate EF_CT_P and EF_CT_PE based on P and PE 
        ## equation 2.5.1-1 and 2.5.1-2
        self.EF_CT_P = math.exp(0.00558 * self.P - 7.7)
        self.EF_CT_PE = math.exp(0.00558 * self.PE - 7.7)

        return self.EF_CT_P, self.EF_CT_PE
    
    def calculate_ef_topo(self):
        ## calculate EF_Topo
        ## equation 2.5.2-1, 2.5.2-2, and 2.5.2-3
        ## ensure EF_CT_P and EF_CT_PE are calculated
        if not hasattr(self, 'EF_CT_P') or not hasattr(self, 'EF_CT_PE'):
            self.calculate_ef_ct()

        intermediate_factor = self.P/self.PE

        if intermediate_factor > 1:
            self.EF_Topo = self.EF_CT_P
        elif self.P == self.PE:
            self.EF_Topo = self.EF_CT_PE
        else:
            self.EF_Topo = ((self.EF_CT_PE * self.FR_Topo/100) + (self.EF_CT_P * (1 - self.FR_Topo/100)))

        return self.EF_Topo
    
    def calculate_emission_factor(self):
        ## calculate emission factor
        ## equation 2.5.3-2 and 2.5.4-1
        ## ensure EF_CT_P and EF_CT_PE are calculated
        if not hasattr(self, 'EF_Topo'):
            self.calculate_ef_topo()
        
        EF_base = (self.EF_Topo*self.RF_TX) * (1/0.645)

        self.EF = EF_base * self.RF_NS * self.RF_till * self.RF_CS * self.RF_AM

        return self.EF
            
    def get_ef(self):
        if not hasattr(self, 'EF'):
            self.calculate_emission_factor()

        self.data_dict = {'EF_CT_P': self.EF_CT_P,
                          'EF_CT_PE': self.EF_CT_PE,
                          'EF_Topo': self.EF_Topo,
                          'EF': self.EF}
        
        return self.data_dict


In [9]:
test_data = {
            'climate_data' : {
                'P': 159,
                'PE': 678,
                'FR_Topo': 7.57
            },
            'modifiers' : {
                'RF_TX': 1,
                'RF_NS': 0.84,
                'RF_Till': 1,
                'RF_CS': 1,
                'RF_AM': 1
            }
}

ef_calculator = EmissionFactorCalculator(test_data)

print('EF_Topo for the farm: ', ef_calculator.calculate_ef_topo())
print('Emission Factor for the farm: ', ef_calculator.calculate_emission_factor())

test_farm_ef = ef_calculator.get_ef()
test_farm_ef

EF_Topo for the farm:  0.0025232347031386142
Emission Factor for the farm:  0.0032860731017619158


{'EF_CT_P': 0.001099631670775916,
 'EF_CT_PE': 0.019905484145844587,
 'EF_Topo': 0.0025232347031386142,
 'EF': 0.0032860731017619158}

## 4. Calculate CRN nitrogen emission 

In [13]:
class EmissionCalculator:
    def __init__(self, ef_data, n_data):
        ## validate input data
        self.validate_input(ef_data, n_data)
        ## initialize instance variables with validated data
        self.ef_data = ef_data
        self.n_data = n_data
        self.EF = ef_data['EF']
        self.n_crop_residue = n_data['n_crop_residue']
        
    
    def validate_input(self, ef_data, n_data):
        ## data should include all required fields and should be numbers
        required_ef_key = ['EF']
        for key in required_ef_key:
            if key not in ef_data:
                raise ValueError(f"Missing required key: {key}")
            if not isinstance(ef_data[key], (int, float)):
                raise TypeError(f"Value for {key} must be a number (int or float)")
            
        required_n_keys = ['n_crop_residue']
        for key in required_n_keys:
            if key not in n_data:
                raise ValueError(f"Missing required key: {key}")
            if not isinstance(n_data[key], (int, float)):
                raise TypeError(f"Value for {key} must be a number (int or float)")            
            
            
    def calculate_n_crn_direct(self):
        ## calculate n_crn_direct
        ## equation 2.6.5-2
        self.n_crn_direct = self.n_crop_residue * self.EF

        return self.n_crn_direct
    
    def calculate_n_other_direct(self):
        ## set values for other sources of n_direct
        ## currently set to 0, cen be modified to include other sources
        self.n_sn_direct = 0
        self.n_crnmin_direct = 0
        self.n_on_direct = 0
    
    def calculate_n_crop_direct(self):
        ## calculate n_crop_direct
        ## equation 2.6.9-1
        if not hasattr(self, 'n_sn_direct') or not hasattr(self, 'n_crnmin_direct') or not hasattr(self, 'n_on_direct'):
            self.calculate_n_other_direct()
        if not hasattr(self, 'n_crn_direct'):
            self.calculate_n_crn_direct()

        self.n_crop_direct = self.n_sn_direct + self.n_crnmin_direct + self.n_on_direct + self.n_crn_direct

        return self.n_crop_direct
    
    def convert_n_crop_direct_to_n2o(self):
        ## calculate n02_crop_direct using n_crop_direct
        ## using constant 44/28
        if not hasattr(self, 'n_crop_direct'):
            self.calculate_n_crop_direct()

        N_TO_NO2 = 44/28

        self.no2_crop_direct = self.n_crop_direct * N_TO_NO2

        return self.no2_crop_direct
    
    def calculate_n2o_crop_direct_to_co2e(self):
        ## calculate co2 equavilent for n2o_crop_direct
        ## using constant 273, the same value used by Holos
        if not hasattr(self, 'no2_crop_direct'):
            self.convert_n_crop_direct_to_n2o()

        NO2_TO_CO2 = 273

        self.co2_crop_direct = self.no2_crop_direct * NO2_TO_CO2

        return self.co2_crop_direct
    
    def get_emission(self):
        if not hasattr(self, 'co2_crop_direct'):
            self.calculate_n2o_crop_direct_to_co2e()

        self.data_dict = {
            'n_crop_direct': self.n_crop_direct,
            'no2_crop_direct': self.no2_crop_direct,
            'co2_crop_direct': self.co2_crop_direct
        }

        return self.data_dict



In [14]:
# test_data = {
#     'n_crop_residue': 560.2468642105264,
#     'EF': 0.0032860731017619158
# }

test_farm_ef
test_farm_n = {
    'n_crop_residue': 560.2468642105264
}

emission_calculator = EmissionCalculator(test_farm_ef, test_farm_n)

print('Crop N direct: ', emission_calculator.calculate_n_crop_direct())
print('Crop NO2 direct: ', emission_calculator.convert_n_crop_direct_to_n2o())
print('Crop CO2e direct: ', emission_calculator.calculate_n2o_crop_direct_to_co2e())

test_farm_emission = emission_calculator.get_emission()
test_farm_emission

Crop N direct:  1.8410121508286712
Crop NO2 direct:  2.8930190941593406
Crop CO2e direct:  789.7942127055


{'n_crop_direct': 1.8410121508286712,
 'no2_crop_direct': 2.8930190941593406,
 'co2_crop_direct': 789.7942127055}